## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv('/Users/malho/Desktop/Analysis_projects/World_Weather_Analysis/Weather_Database/WeatherPy_Database.csv')
#pd.read_csv("Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Kavieng,PG,-2.5744,150.7967,83.07,73,96,14.25,overcast clouds
1,1,Longyearbyen,SJ,78.2186,15.6401,22.84,74,20,4.61,few clouds
2,2,Goderich,CA,43.7501,-81.7165,30.90,68,82,13.33,broken clouds
3,3,Mar Del Plata,AR,-38.0023,-57.5575,59.88,93,75,8.01,fog
4,4,Kuala Sungai Baru,MY,2.3594,102.0353,78.82,88,100,10.47,light rain


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
filter_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]
filter_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Kavieng,PG,-2.5744,150.7967,83.07,73,96,14.25,overcast clouds
4,4,Kuala Sungai Baru,MY,2.3594,102.0353,78.82,88,100,10.47,light rain
5,5,Rikitea,PF,-23.1203,-134.9692,77.77,67,18,11.63,few clouds
6,6,Maun,BW,-19.9833,23.4167,85.59,36,84,7.20,broken clouds
8,8,Noumea,NC,-22.2763,166.4572,79.00,99,100,1.81,overcast clouds
10,10,Salalah,OM,17.0151,54.0924,87.89,70,20,13.80,few clouds
11,11,San Patricio,US,28.0170,-97.5169,77.09,97,4,7.09,clear sky
13,13,Tessalit,ML,20.1986,1.0114,84.61,12,2,9.69,clear sky
14,14,Vaini,TO,-21.2000,-175.2000,81.18,79,90,14.12,overcast clouds
17,17,Merauke,ID,-8.4667,140.3333,81.21,79,100,4.99,overcast clouds


In [5]:
# 4a. Determine if there are any empty rows.
filter_cities_df.isnull().sum()

City_ID                0
City                   0
Country                4
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = filter_cities_df.dropna()
clean_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Kavieng,PG,83.07,overcast clouds,-2.5744,150.7967,
4,Kuala Sungai Baru,MY,78.82,light rain,2.3594,102.0353,
5,Rikitea,PF,77.77,few clouds,-23.1203,-134.9692,
6,Maun,BW,85.59,broken clouds,-19.9833,23.4167,
8,Noumea,NC,79.00,overcast clouds,-22.2763,166.4572,
10,Salalah,OM,87.89,few clouds,17.0151,54.0924,
11,San Patricio,US,77.09,clear sky,28.0170,-97.5169,
13,Tessalit,ML,84.61,clear sky,20.1986,1.0114,
14,Vaini,TO,81.18,overcast clouds,-21.2000,-175.2000,
17,Merauke,ID,81.21,overcast clouds,-8.4667,140.3333,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [15]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df['Hotel Name'].replace('', np.nan, inplace=True)
clean_hotel_df = hotel_df.dropna()
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Kavieng,PG,83.07,overcast clouds,-2.5744,150.7967,Nusa Island Retreat
4,Kuala Sungai Baru,MY,78.82,light rain,2.3594,102.0353,Homestay Villa Suraiya Kuala Sungai Baru Melaka
5,Rikitea,PF,77.77,few clouds,-23.1203,-134.9692,People ThankYou
6,Maun,BW,85.59,broken clouds,-19.9833,23.4167,Enviro Villa
8,Noumea,NC,79.00,overcast clouds,-22.2763,166.4572,Hôtel Le Lagon
...,...,...,...,...,...,...,...
677,Labuhan,ID,83.14,broken clouds,-6.8844,112.2051,DoubleFAG
679,Oranjestad,AW,80.55,few clouds,12.5240,-70.0270,Divi Village Golf & Beach Resort
686,Labuan,MY,84.56,few clouds,5.2767,115.2417,Tiara Labuan Hotel
688,Flic En Flac,MU,83.52,light rain,-20.2742,57.3631,Hilton Mauritius Resort & Spa


In [16]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [17]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [18]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))